In [10]:
import sys
import logging

In [11]:
class Expense():
    def __init__(self, target_year, amount):
        self.target_year = target_year
        self.amount = amount

In [12]:
class SavingsGoal():
    def __init__(self, start_year, end_year, amount):
        self.start_year = start_year
        self.end_year = end_year
        self.amount = amount
    
    def get_savings_rate_year(self):
        return self.amount / (self.end_year - self.start_year)
    
    def get_savings_rate_month(self):
        return self.get_savings_rate_year() / 12
    
    def __repr__(self):
        return "[{start}-{end}: {rate_month: >6.0f} kr/month {rate_year: >8.0f} kr/year {amount: >8} kr total]".format(
            start=self.start_year,
            end=self.end_year,
            amount=self.amount,
            rate_year=self.get_savings_rate_year(),
            rate_month=self.get_savings_rate_month()
        )

In [13]:
def print_savings_goals(savings_goals):
    print("--- Savings goals ---")
    savings_goal_index = 0
    start_year = savings_goals[0].start_year
    end_year = savings_goals[-1].end_year
    for year in range(start_year, end_year):
        if savings_goal_index+1 < len(savings_goals) and year == savings_goals[savings_goal_index+1].start_year:
            savings_goal_index += 1
        savings_goal = savings_goals[savings_goal_index]
        
        print("{year}: {savings_rate_month:.0f} kr/month ({savings_rate_year:.0f} kr/year)".format(
            year=year,
            savings_rate_month=savings_goal.get_savings_rate_month(),
            savings_rate_year=savings_goal.get_savings_rate_year()
        ))
        if year == savings_goal.end_year:
            savings_goal_index += 1

In [14]:
def print_time_line(savings_goals, expenses):
    savings_goal_index = 0
    start_year = savings_goals[0].start_year
    year = start_year
    #total_savings = [(start_year, 0)]
    balance = 0

    while year <= savings_goals[-1].end_year:
        print(year)
        print("----")
        print("Start of year balance: {:.0f} kr".format(balance))
        
        if savings_goal_index+1 < len(savings_goals) and year == savings_goals[savings_goal_index+1].start_year:
            print("Costs: {} kr".format(current_expense.amount))
            balance -= current_expense.amount
            print("Balance after costs: {:.0f} kr".format(balance))
            savings_goal_index += 1
        elif savings_goal_index == len(savings_goals)-1:
            print("Costs: {} kr".format(current_expense.amount))
            balance -= current_expense.amount
            print("Balance after costs: {:.0f} kr".format(balance))
            break
        
        current_savings_goal = savings_goals[savings_goal_index]
        current_expense = expenses[savings_goal_index]
        
        print("Saving {:.0f} kr".format(current_savings_goal.get_savings_rate_year()))
        balance += current_savings_goal.get_savings_rate_year()
        
        year += 1
        print()

In [15]:
STARTING_YEAR = 2021
expenses = [
    Expense(2022, 4000+5000), # uterum + barn rum
    Expense(2024, 150000), # badrum nere
    Expense(2025, 140000), # tvättstuga
    Expense(2027, 220000), # kök
    Expense(2028, 70000+10000), # tak + trappa/hall
    Expense(2029, 100000+7000), # bil + allrum
    Expense(2030, 125000), # fönster/dörrar
    Expense(2031, 50000), # måla fasad
    Expense(2033, 150000), # badrum uppe
    Expense(2036, 40000), # fjärrvärme
    Expense(2037, 30000), # FTX aggregat
]
logging.basicConfig(level=logging.INFO)

In [16]:
# init savings goals
savings_goals = []
for i, expense in enumerate(expenses):
    previous_expense_target_year = STARTING_YEAR if i == 0 else expenses[i-1].target_year
    savings_goal = SavingsGoal(previous_expense_target_year, expense.target_year, expense.amount)
    savings_goals.append(savings_goal)

In [17]:
print("\n".join([str(g) for g in savings_goals]))

[2021-2022:    750 kr/month     9000 kr/year     9000 kr total]
[2022-2024:   6250 kr/month    75000 kr/year   150000 kr total]
[2024-2025:  11667 kr/month   140000 kr/year   140000 kr total]
[2025-2027:   9167 kr/month   110000 kr/year   220000 kr total]
[2027-2028:   6667 kr/month    80000 kr/year    80000 kr total]
[2028-2029:   8917 kr/month   107000 kr/year   107000 kr total]
[2029-2030:  10417 kr/month   125000 kr/year   125000 kr total]
[2030-2031:   4167 kr/month    50000 kr/year    50000 kr total]
[2031-2033:   6250 kr/month    75000 kr/year   150000 kr total]
[2033-2036:   1111 kr/month    13333 kr/year    40000 kr total]
[2036-2037:   2500 kr/month    30000 kr/year    30000 kr total]


In [9]:
logging.debug("Starting calibration of saving rate\n")
while True:
    transfer_made = False
    
    # loop over all savings goals in chronological order
    for i, savings_goal in enumerate(savings_goals):
        logging.debug("Calibrating savings goal {}".format(savings_goal))
        
        # get the savings goal previous to the current that has the lowest "saving amount per month"
        prev_savings_goals = savings_goals[:i]
        logging.debug("Previous savings goals: \n{}".format("\n".join([str(g) for g in prev_savings_goals])))
        
        min_amount_per_year = sys.maxsize
        min_amount_per_year_savings_goal = None
        for prev_savings_goal in prev_savings_goals:
            if prev_savings_goal.get_savings_rate_year() < savings_goal.get_savings_rate_year() and prev_savings_goal.get_savings_rate_year() < min_amount_per_year:
                min_amount_per_year = prev_savings_goal.get_savings_rate_year()
                min_amount_per_year_savings_goal = prev_savings_goal
             
        # calculate the amount the transfer to the previous savings goal to even out the "saving amount per month"
        if min_amount_per_year_savings_goal is not None:
            transfer_made = True
            logging.debug("Previous savings goal with minimum savings amount: {}".format(str(min_amount_per_year_savings_goal)))
            # transfer_amount = (savings_goal.get_savings_amount_per_year() - min_amount_per_year_savings_goal.get_savings_amount_per_year()) / (1/min_amount_per_year_savings_goal.nr_years_saving_period + 1/savings_goal.nr_years_saving_period)
            transfer_amount = 10
            logging.debug("Amount to transfer: {}".format(transfer_amount))
            savings_goal.amount = savings_goal.amount - transfer_amount
            logging.debug("New target amount for savings goal : {}".format(savings_goal.amount))
            min_amount_per_year_savings_goal.amount = min_amount_per_year_savings_goal.amount + transfer_amount
            logging.debug("New target amount for previous savings goal : {}".format(min_amount_per_year_savings_goal.amount))
        else:
            logging.debug("Found no previous savings goals with lesser savings amount per year than me")
        logging.debug("---\n")
    
    if not transfer_made:
        # stop iterating
        break

print_savings_goals(savings_goals)
print("\n")
print_time_line(savings_goals, expenses)

--- Savings goals ---
2021: 7991 kr/month (95890 kr/year)
2022: 7991 kr/month (95890 kr/year)
2023: 7991 kr/month (95890 kr/year)
2024: 7991 kr/month (95890 kr/year)
2025: 7726 kr/month (92710 kr/year)
2026: 7726 kr/month (92710 kr/year)
2027: 7550 kr/month (90600 kr/year)
2028: 7549 kr/month (90590 kr/year)
2029: 7549 kr/month (90590 kr/year)
2030: 5308 kr/month (63700 kr/year)
2031: 5308 kr/month (63690 kr/year)
2032: 5308 kr/month (63690 kr/year)
2033: 1441 kr/month (17290 kr/year)
2034: 1441 kr/month (17290 kr/year)
2035: 1441 kr/month (17290 kr/year)
2036: 1441 kr/month (17290 kr/year)


2021
----
Start of year balance: 0 kr
Saving 95890 kr

2022
----
Start of year balance: 95890 kr
Costs: 9000 kr
Balance after costs: 86890 kr
Saving 95890 kr

2023
----
Start of year balance: 182780 kr
Saving 95890 kr

2024
----
Start of year balance: 278670 kr
Costs: 150000 kr
Balance after costs: 128670 kr
Saving 95890 kr

2025
----
Start of year balance: 224560 kr
Costs: 140000 kr
Balance after